In [ ]:
!rm -rf 'gym-hnef'

In [ ]:
!git clone https://github.com/calenirwin/gym-hnef
!pip install -e gym-hnef

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import gym
from gym_hnef import hnef_game, hnef_vars
from gym_hnef.envs import hnef_env
import model
import mcts
import agent
import config
import funcs
import memory

In [ ]:
# 9 * 9 * 9 * 9, size of the action space
action_size = 6561
# initial state of the game
init_state = hnef_game.init_state('historical')

In [ ]:
# Create the neural networks
current_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)
best_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)

In [ ]:
# Create the players
current_player = agent.Agent('current_player', current_NN, init_state.shape, action_size)
best_player = agent.Agent('best_player', best_NN, init_state.shape, action_size)

In [ ]:
iteration = 0

max_iter = 1e2

mem = memory.Memory(config.MEMORY_SIZE)

version = 0

while iteration < max_iter:
    # Self play
    scores, mem = funcs.play_matches(best_player, best_player, mem=mem)
    print(scores)

    mem.clear_stmemory()

    if len(mem.ltmemory) >= config.MEMORY_SIZE:

        current_player.replay(mem.ltmemory)

        scores, _ = funcs.play_matches(best_player, current_player)
        print(scores)

        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            version += 1
            print(version)
            best_NN.model.set_weights(current_NN.model.get_weights())
    else:
        print('Memory Usage: ', str(len(mem.ltmemory)), "/", str(config.MEMORY_SIZE))
    iteration += 1

In [ ]:
random_player = agent.Agent('random_player', init_state.shape, action_size)
scores, all_end_states = funcs.evaluate_agents(best_player, random_player)

In [ ]:
#current_NN.model.save_weights('best_weights.h5')